# March Machine Learning Mania 2016 - Kaggle 
For my first Kaggle competition, I tried my hand at March Madness!
We were provided up to three decades of college basketball statistics, as well as encouraged to use external data, to predict game outcomes. 

## Initializations and Reusable Functions

In [1]:
#Show any plotted graphs within the notebook
% matplotlib inline

import pandas as pd #Data manipulation library
import numpy as np #Numerical analysis and linear algebra library

# Matplotlib provides the ability to plot figures and charts in python
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt

# sklearn provides various scientific functionality as well as machine learning models for data analysis
import sklearn 

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

# miscellaneous libraries to help keep track of code run time, various math operations etc
import datetime
import time
import math


### Functions :
**1) doEverything ** <br>
**2) winProb **

In [2]:
def doEverything(training, testing, predictCols, year, calcLogLoss = False ):

    yValues = training[training.season<=year]['outcome']
    xValues = preprocessing.scale(training[training.season<=year][predictCols])

    clf=LogisticRegression()
    parameters = {"C": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
    fitmodel = GridSearchCV(clf, param_grid=parameters, cv=5, scoring="log_loss")
    fitmodel.fit(xValues, yValues)
    # fitmodel.best_estimator_, fitmodel.best_params_, fitmodel.best_score_, fitmodel.grid_scores_
    
    # use the preprocessing tools from SKlearn to scale our test features - center around mean, scale by variance
    results = fitmodel.predict_proba(preprocessing.scale(testing[testing.season==year][predictCols]))
    testing.ix[testing.season==year, 'Pred'] = results[:,1]  #Get the second column which is the prob of 1 (victory)
    
    if calcLogLoss:
        # if you want to calculate the log_loss of the given season
        # find the win probabilities that were calculated for each winning team by your model
        # then calculate the log_loss
        TourneyDet['wprob'] = TourneyDet.apply(winProb, axis = 1)
        TourneyDet['logloss'] = np.log(TourneyDet['wprob'].values)*-1
        return TourneyDet[TourneyDet['season']==year]['logloss'].mean()
    

In [3]:
def winProb(row):
    # from historical tournament data, given a Season and the winning/losing teams in a particular match-up
    # find the predicted win-probability the model estimated for the team that actually won
    chosenProb = SeasonAvg[(SeasonAvg.season == row.season) & (SeasonAvg.team1==min(row.wteam, row.lteam)) & 
                           (SeasonAvg.team2== max(row.lteam,row.wteam))]['Pred'].sum()
    if row.wteam > row.lteam:
        return (1 - chosenProb)
    return chosenProb

# Explore Data

Data that went all the way back thirty years was rather compact and provided high level metrics such as points, winner/loser, game location, and number of overtimes. <br> 
The more granular data was relatively shorter dated (going back only thirteen years), including things such as points, rebounds, steals, blocked shots, assists, turnovers, and fouls for each game played. <br>
Kaggle additionally provided an amalgamation of team rankings from professional sports-media outlets for each season.

In [4]:
#read in the files of data
#TourneyCompactResults = pd.read_csv("data/TourneyCompactResults.csv")  #1985-2015 tourneys with basic win/loss details
#TourneySeeds = pd.read_csv("data/TourneySeeds.csv")   #Team seeds for 1985-2015 
#MasseyOld = pd.read_csv("data/massey_ordinals_2003-2015.csv") #HIST collection of ~60 rankings across various analysts

TourneyDet = pd.read_csv("data/TourneyDetailedResults.csv")  #HIST TOURNEY data in detail, used in calcing log loss at the end
TourneyDet.columns = [col.lower() for col in TourneyDet.columns]  #Make all cols lower case so various functions work

RegDet = pd.read_csv("data/RegularSeasonDetailedRanked.csv")# Detailed regular season data updated with team ranks

Latest = pd.read_csv("data/RegularSeasonDetailedResults.csv") #Detailed, Regular season data
Latest.columns = [col.lower() for col in Latest.columns]

# ~60 diff sports analysts providing current ranks for each team
Massey = pd.read_csv("data/MasseyOrdinals2016ThruDay133_59systems.csv")  

SampleSubmission = pd.read_csv("data/SampleSubmission.csv")  #Read sample submit file for all matchups and win probs

From the latest available regular season data that includes historical seasons as well as the current 2015-2016 season, we create a new dataframe containing only the current season stats. We'll manipulate this data and ultimately make predictions on them.

In [5]:
RegDet2016 = Latest[Latest.season==2016].copy()
RegDet2016.head()

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,...,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
65872,2016,11,1104,77,1244,64,H,0,29,57,...,19,19,26,12,27,6,16,7,4,25
65873,2016,11,1105,68,1408,67,A,1,25,64,...,27,16,26,18,30,11,19,6,7,21
65874,2016,11,1112,79,1334,61,H,0,24,61,...,19,13,19,5,23,9,9,3,1,25
65875,2016,11,1115,58,1370,56,A,0,20,55,...,16,16,28,10,31,12,15,5,0,17
65876,2016,11,1116,86,1380,68,H,0,32,66,...,12,20,28,7,21,9,17,8,5,22


Apply the average Sports-Media rankings (compiled by Kenneth Massey) to each winning/losing team on each respective season and day

In [6]:
#While iterating through a given dataframe's rows:
#Group Massey ranking data by current row's: W/L team id, season, and day of season.
#Return the average 'ranking' (orank field) of this grouped data.


def getAvgRankL(row):
    if int(row.name)%1000==0:
        print row.name, #prints current row number of DF to track progress
    return Massey[(Massey.team == row['lteam']) & (Massey.season == row['season']) & (Massey.rating_day_num == row['daynum'])].orank.mean()
def getAvgRankW(row):
    if int(row.name)%1000==0:
        print row.name, #prints current row number of DF to track progress
    return Massey[(Massey.team == row['wteam']) & (Massey.season == row['season']) & (Massey.rating_day_num == row['daynum'])].orank.mean()

In [7]:
##Apply the average Kenneth Massey rankings to each winning/losing team on each respective season and day

###Old code used to create the 'RegularSeasonRanked' file from historical data while i was building this model
#RegDet['wrank'] = RegDet.apply(getAvgRankW, axis=1) 
#RegDet['lrank'] = RegDet.apply(getAvgRankL, axis=1) 

RegDet2016['wrank'] = RegDet2016.apply(getAvgRankW, axis=1) 
RegDet2016['lrank'] = RegDet2016.apply(getAvgRankL, axis=1) 

#Concatenate 2016 season rankings with previous seasons rankings
RegDet = pd.concat([RegDet,RegDet2016], ignore_index = True)


##Once you are done assigning the average rankings to each winning and losing team, preserve the dataframe to CSV file
#RegDet.to_csv('data/RegularSeasonDetailedRanked.csv', index = False)

66000 67000 68000 69000 70000 71000 66000 67000 68000 69000 70000 71000


In [8]:
#print out the combined RegDet dataframe to make sure the combination went smoothly
RegDet.head()

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,...,lfta,lor,ldr,last,lto,lstl,lblk,lpf,wrank,lrank
0,2003,10,1104,68,1328,62,N,0,27,58,...,22,10,22,8,18,9,2,20,10.000000,140.000000
1,2003,10,1272,70,1393,63,N,0,26,62,...,20,20,25,7,12,8,6,16,35.750000,62.250000
2,2003,11,1266,73,1437,61,N,0,24,58,...,23,31,22,9,12,2,5,23,52.000000,99.750000
3,2003,11,1296,56,1457,50,N,0,18,38,...,15,17,20,9,19,4,3,23,226.684211,94.157895
4,2003,11,1400,77,1208,71,N,0,30,61,...,27,21,15,12,10,7,1,14,12.750000,209.250000


In [9]:
RegDet.tail()

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,...,lfta,lor,ldr,last,lto,lstl,lblk,lpf,wrank,lrank
71236,2016,132,1114,70,1419,50,N,0,26,52,...,9,6,20,13,13,6,3,12,NaN,NaN
71237,2016,132,1163,72,1272,58,N,0,22,48,...,17,5,21,10,6,3,0,20,NaN,NaN
71238,2016,132,1246,82,1401,77,N,1,28,58,...,22,17,23,11,13,5,4,20,NaN,NaN
71239,2016,132,1277,66,1345,62,N,0,25,60,...,21,5,22,10,5,4,4,14,NaN,NaN
71240,2016,132,1386,87,1433,74,N,0,35,54,...,16,12,21,12,9,5,5,21,NaN,NaN


Using a ranking system from external sources (despite being provided by Kaggle) leads to the problem of inconsistent availability (ie. ranks were only updated on a periodic basis therefore on certain game-days a NaN value was populated, some teams didn't get ranked in particular years, or if a new team had been added to the Division 1 NCAA competition pool). We use the fill() functions offered in Pandas to quickly and (relatively) intelligently populate missing data.

In [10]:
#Based on each team in a given season:
#first we remove the NaNs for all Winning teams by using "forwardFill" to apply the latest avgRank going forward 
RegDet['wrank'] = RegDet.groupby(['season','wteam']).wrank.ffill()   
#second, we remove the remaining NaNs at beginning of time series, using "backFill", applying first available rank backwards 
RegDet['wrank'] = RegDet.groupby(['season','wteam']).wrank.bfill()
#Do this same process on the losing teams' ranks
RegDet['lrank'] = RegDet.groupby(['season','wteam']).lrank.ffill()   
RegDet['lrank'] = RegDet.groupby(['season','wteam']).lrank.bfill()

#If there are any teams that didn't have rankings in a particular season. Simply take the previous years 
#ranks and forward fill them  (If no prior data exists for this team, then backfill the first available data)
RegDet['wrank'] = RegDet.groupby('wteam').wrank.ffill()
RegDet['wrank'] = RegDet.groupby('wteam').wrank.bfill()
RegDet['lrank'] = RegDet.groupby('lteam').lrank.ffill()
RegDet['lrank'] = RegDet.groupby('lteam').lrank.bfill()

#If there is a new team that hasn't been ranked at all, apply the mean rank of the winning/losing category
RegDet['wrank'] = RegDet.wrank.fillna(RegDet.wrank.mean())
RegDet['lrank'] = RegDet.lrank.fillna(RegDet.lrank.mean())


In [11]:
#Do a quick check to see if our NaNs in rank fields were properly cleaned up and explore the ones that are still NaN
#RegDet[(RegDet['wrank'].isnull()) & (RegDet['season']>=2003)]
#RegDet[(RegDet['wteam'] == 1213) & (RegDet['season']>2003)]
(RegDet.isnull()*1).sum()   #Check to see if any columns have NaN values

season    0
daynum    0
wteam     0
wscore    0
lteam     0
lscore    0
wloc      0
numot     0
wfgm      0
wfga      0
wfgm3     0
wfga3     0
wftm      0
wfta      0
wor       0
wdr       0
wast      0
wto       0
wstl      0
wblk      0
wpf       0
lfgm      0
lfga      0
lfgm3     0
lfga3     0
lftm      0
lfta      0
lor       0
ldr       0
last      0
lto       0
lstl      0
lblk      0
lpf       0
wrank     0
lrank     0
dtype: int64

In [12]:
#Convert winners' qualitative homecourt status into quantitative values: Away = 0, Neutral = 1, Home = 2
print "Before: %r" % RegDet['wloc'].dtype
RegDet['wloc'].replace({'A':0, 'N': 1, 'H': 2}, inplace = True)
print "After: %r" % RegDet['wloc'].dtype
#Check to see if you converted all the 'object' values in game location to an int


Before: dtype('O')
After: dtype('int64')


When we get a sense for how a particular team compares to its opponent, we can use these statistical differences as new 'features' for model building purposes:

In [13]:
#Calculate differences in each of the statistics between winning or losing teams
for col in RegDet.columns[3:]:
        if col.startswith('w'):
            NewColName1 = "%sdiff" % col
            LColName = "l%s" % col[1:]
            NewColName2 = "%sdiff" % LColName
            if (RegDet.columns[3:]).isin([LColName]).sum():
                #print LColName
                #print NewColName1
                #print NewColName2
                RegDet[NewColName1] = RegDet[col] - RegDet[LColName]
                RegDet[NewColName2] = RegDet[LColName] - RegDet[col]

#Create columns of 1 or 0 for 'winning' and 'losing' teams respectively
RegDet['woutcome'] = 1
RegDet['loutcome'] = 0

Currently, the data is spread out horizontally (going left to right, from winners to losers), but ultimately we would like to stack them vertically such that the win/loss ("outcome") column can be one array of input into model training.<br>
So we split winners and losers apart, rename their columns to match each other (dropping the "w"/"l"), and re-join them vertically.

In [14]:
#Extract the Winners and Losers into two separate sets of data
season = ['season']
winners = season +([col for col in RegDet.columns if col.startswith('w')])
losers = season + ([col for col in RegDet.columns if col.startswith('l')])
winningSet = RegDet[winners]
losingSet = RegDet[losers]

#Remove the "w" and "l" indicators in the column names, if they exist
newNamesW = season + ([col[1:] for col in winningSet.columns if col.startswith('w')])
newNamesL = season + ([col[1:] for col in losingSet.columns if col.startswith('l')])
winningSet.rename(columns = dict(zip(winners, newNamesW)), inplace = True)
losingSet.rename(columns = dict(zip(losers, newNamesL)), inplace = True)

/Users/ankitpatel/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [15]:
winningSet.head()

,season,team,score,loc,fgm,fga,fgm3,fga3,ftm,fta,...,ftadiff,ordiff,drdiff,astdiff,todiff,stldiff,blkdiff,pfdiff,rankdiff,outcome
0,2003,1104,68,1,27,58,3,14,11,18,...,-4,4,2,5,5,-2,-1,2,-130.000000,1
1,2003,1272,70,1,26,62,8,20,10,19,...,-1,-5,3,9,1,-4,-2,2,-26.500000,1
2,2003,1266,73,1,24,58,8,18,17,29,...,6,-14,4,6,-2,3,-3,2,-47.750000,1
3,2003,1296,56,1,18,38,3,9,17,31,...,16,-11,-1,2,-7,10,-1,-5,132.526316,1
4,2003,1400,77,1,30,61,6,14,11,13,...,-14,-4,7,0,4,-3,3,6,-196.500000,1


In [16]:
losingSet.head()

,season,team,score,fgm,fga,fgm3,fga3,ftm,fta,or,...,ftadiff,ordiff,drdiff,astdiff,todiff,stldiff,blkdiff,pfdiff,rankdiff,outcome
0,2003,1328,62,22,53,2,10,16,22,10,...,4,-4,-2,-5,-5,2,1,-2,130.000000,0
1,2003,1393,63,24,67,6,24,9,20,20,...,1,5,-3,-9,-1,4,2,-2,26.500000,0
2,2003,1437,61,22,73,3,26,14,23,31,...,-6,14,-4,-6,2,-3,3,-2,47.750000,0
3,2003,1457,50,18,49,6,22,8,15,17,...,-16,11,1,-2,7,-10,1,5,-132.526316,0
4,2003,1208,71,24,62,6,16,17,27,21,...,14,4,-7,0,-4,3,-3,-6,196.500000,0


In [17]:
#concatinate the winning and losing sets of data so that you have a series of wins/loses (ie 1s/0s)
#and the corresponding statistics, as features, to go with them
#These features will be used to create a logistic model to predict probability of winning (1)
compiledList = pd.concat([winningSet,losingSet],ignore_index = True)  #Ignore index is used to create a new index col
compiledList.head()

,ast,astdiff,blk,blkdiff,dr,drdiff,fga,fga3,fga3diff,fgadiff,...,rank,rankdiff,score,scorediff,season,stl,stldiff,team,to,todiff
0,13,5,1,-1,24,2,58,14,4,5,...,10.000000,-130.000000,68,6,2003,7,-2,1104,23,5
1,16,9,4,-2,28,3,62,20,-4,-5,...,35.750000,-26.500000,70,7,2003,4,-4,1272,13,1
2,15,6,2,-3,26,4,58,18,-8,-15,...,52.000000,-47.750000,73,12,2003,5,3,1266,10,-2
3,11,2,2,-1,19,-1,38,9,-13,-11,...,226.684211,132.526316,56,6,2003,14,10,1296,12,-7
4,12,0,4,3,22,7,61,14,-2,-1,...,12.750000,-196.500000,77,6,2003,4,-3,1400,14,4


Now let's set ourselves up with a dataframe ready to make predictions. The SampleSubmission dataframe gives us a list of every possible match-up we want to make predictions for, but the season and team numbers are all represented as one string. Let's split out each element in the "Id" column (creating three new columns, which individually represent: Season, Team1, and Team2).

In [18]:
#Split out the game match-up string into new columns:  Season, Team 1, and Team 2. Convert these columns to data type 'int'
#extract a new set of columns from testDF to calculate "avgStats" for Teams 1 and 2
SeasonAvg = SampleSubmission.copy()
testDF = compiledList.drop(['season','team','loc', 'outcome'], axis = 1)

newCols = SeasonAvg['Id'].str.rsplit('_', expand = True)
SeasonAvg['season'], SeasonAvg['team1'], SeasonAvg['team2'] = newCols[0],newCols[1],newCols[2]
SeasonAvg[['season', 'team1', 'team2']] = SeasonAvg[['season', 'team1', 'team2']].astype(int)

avgStats = [col for col in testDF.columns if not(col.endswith('diff'))]
diffStats = [col + "diff" for col in avgStats]



Now that we've broken out the SampleSubmission dataframe into the individual components that we care about, we need to populate this new dataframe with relevant statistical data (features) so that we can make predictions. <br>
The two mini functions below (getAvgStats and getStatsDiff) will be used to go through each row of the new SeasonAvg dataframe and extract the subsets of data that are applicable to the given row's season and teams.

In [19]:
#Given a SEASON and TEAM, compute what the average stats would be at the end of that season
def getAvgStats(row):
    return compiledList[(compiledList['season'] == row['season']) & (compiledList['team'] == row['team1'])][avgStats].mean()

#Then get the DIFFERENCE of these average end-of-season stats, between two teams in the sample submission file 
def getStatsDiff(row):
    Team1Data = compiledList[(compiledList['season'] == row['season']) & (compiledList['team'] == row['team1'])][avgStats].mean()
    Team2Data = compiledList[(compiledList['season'] == row['season']) & (compiledList['team'] == row['team2'])][avgStats].mean()
    return (Team1Data-Team2Data)

In [20]:
#For all possible matchups in the tourney, get the avg stats each Team 1 had for the respective season
#Also get the statistical differences against their hypothetical opponent
SeasonAvg[avgStats] = SeasonAvg.apply(getAvgStats, axis=1) 
SeasonAvg[diffStats] = SeasonAvg.apply(getStatsDiff, axis = 1)
SeasonAvg.head()


## If you want to create a stand alone table of the regular season stat averages, grouped by Season and Team number
## This is more for viewing/cross-checking purposes, the 'apply' functions above do the real data manipulation 
## that we will be using.
#groupedAvg = compiledList.groupby(['season','team'])[avgStats].mean()
#groupedAvg.head()
#print groupedAvg.index.values

,Id,Pred,season,team1,team2,ast,blk,dr,fga,fga3,...,fgmdiff,fgm3diff,ftadiff,ftmdiff,ordiff,pfdiff,rankdiff,scorediff,stldiff,todiff
0,2016_1112_1114,0.739194,2016,1112,1114,14.515152,4.787879,28.606061,58.242424,17.030303,...,3.028348,-0.839687,8.567937,5.995112,2.926686,-0.045943,-37.265424,11.212121,-1.381232,2.592375
1,2016_1112_1122,0.967194,2016,1112,1122,14.515152,4.787879,28.606061,58.242424,17.030303,...,2.484848,0.303030,-0.212121,1.212121,0.545455,0.575758,-205.870264,6.484848,-2.000000,-1.727273
2,2016_1112_1124,0.680447,2016,1112,1124,14.515152,4.787879,28.606061,58.242424,17.030303,...,1.248106,0.327652,3.009470,2.075758,-1.926136,-1.194129,-16.678584,4.899621,-2.809659,0.005682
3,2016_1112_1138,0.871170,2016,1112,1138,14.515152,4.787879,28.606061,58.242424,17.030303,...,2.242424,-0.848485,0.878788,0.969697,-0.454545,-3.060606,-99.866627,4.606061,-1.818182,-1.000000
4,2016_1112_1139,0.725995,2016,1112,1139,14.515152,4.787879,28.606061,58.242424,17.030303,...,0.060606,-0.581623,1.793744,1.091887,0.217009,-1.691105,-32.413899,0.631476,-1.865103,2.592375


In [25]:
### Use this loop to calculate win probs for each tourney year and also calculate the logloss errors for each prev
### season
#summary = {}
#for year in [2012, 2013, 2014, 2015]:
#    summary[year] = doEverything(compiledList, SeasonAvg, ['rankdiff', 'blk'], year, True)

### Write your calculated probabilities to csv file for submission to competition
#SampleSubmission['Pred'] = SeasonAvg['Pred']
#Write the data to a file once you've confirmed everything looks okay:
#SampleSubmission.to_csv("data/SampleSubmission.csv", index = False)  
          
#summary  #print out the dictionary of each year's log loss calculation
# TourneyDet[TourneyDet['season']>=2012]['logloss'].mean()

We've now got a collection of historical stats and rankings for winners and losers (compiledList dataframe), upon which we'll train our model, as well as a collection of averaged stats and rankings from the current season (SeasonAvg dataframe), upon which we'll make our predictions. 

In [22]:
### For the new sample submission file, which has matchups for 2016 only:
### Run the model only once
### Assign the values from our temporary DF ("SeasonAvg") to the finalized DF  ("SampleSubmission")
### Write the "SampleSubmission" DF to CSV file for upload

# some prior work was done using an iterative approach to test various combinations of stats that would seem to provide
# the most effective predictors. "Rankdiff" (ranking differential) and "blk" (avg blocks per game) were most effective
doEverything(compiledList, SeasonAvg, ['rankdiff', 'blk'], 2016)
SampleSubmission['Pred'] = SeasonAvg['Pred']
filename = "data/SampleSubmission-%s.csv" % (datetime.datetime.now())
#SampleSubmission.to_csv(filename, index = False)
SampleSubmission.head()

,Id,Pred
0,2016_1112_1114,0.739194
1,2016_1112_1122,0.967194
2,2016_1112_1124,0.680447
3,2016_1112_1138,0.871170
4,2016_1112_1139,0.725995


In [23]:
# check to make sure we aren't missing any predicted values for our submission
(SampleSubmission.isnull()*1).sum()

Id      0
Pred    0
dtype: int64